In [13]:
import numpy as np
from ortools.linear_solver import pywraplp
from scipy.optimize import linprog
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

In [5]:
# Instantiate a GLOP(Google Linear Optimization Package) solver
solver = pywraplp.Solver.CreateSolver('GLOP')

In [6]:
# Create the two variables and let them take on any non-negative value.
x1 = solver.NumVar(0, solver.infinity(), 'x1')
x2 = solver.NumVar(0, solver.infinity(), 'x2')

In [7]:
# Constraint 1: 2x_1 + 5x_2 <= 30.0
solver.Add(2 * x1 + 5 * x2 <= 30.0)

# Constraint 2: 4x_1 + 2x_2 <= 20.0
solver.Add(4 * x1 + 2 * x2 <= 20.0)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000002544399D1A0> >

In [8]:
# Objective function: 3x_1 + 4x_2
solver.Maximize(3 * x1 + 4 * x2)

In [9]:
# Solve the system.
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    print(f'(x1, x2): ({x1.solution_value():.2}, {x2.solution_value():.2})')
else:
    print('The problem does not have an optimal solution.')

Objective value = 27.5
(x1, x2): (2.5, 5.0)


In [3]:
# Construct parameters
c_ex1 = np.array([3, 4])

# Inequality constraints
A_ex1 = np.array([[2, 5],
                  [4, 2]])
b_ex1 = np.array([30,20])

In [4]:
# Solve the problem
# we put a negative sign on the objective as linprog does minimization
res_ex1 = linprog(-c_ex1, A_ub=A_ex1, b_ub=b_ex1)

if res_ex1.success:
    # We use negative sign to get the optimal value (maximized value)
    print('Optimal Value:', -res_ex1.fun)
    print(f'(x1, x2): {res_ex1.x[0], res_ex1.x[1]}')
else:
    print('The problem does not have an optimal solution.')

Optimal Value: 27.5
(x1, x2): (np.float64(2.5), np.float64(5.0))


# Exercise 37.1

Implement a new extended solution for the problem 1 where in the factory owner decides that the number of units of product 1 should not be less than the number of units of product 2.

The problem was:
$$\begin{align}
\text{max } 3x_{1} &+ 4x_{2}  \\
\text{subject to } 2x_{1}&+ 5x_{2} \leq 30 \\
4x_{1} &+ 2x_{2} \leq 20 \\
 x_{1},x_{2} &\geq 0
\end{align}$$

In [11]:
objective = np.array([3,4])
# now we have the additional constraint 0 <= x2 <= x1
# x1 >= x2 => x1 - x2 >= 0 => -x1 + x2 <= 0 (linprog wants <= ineqs)
constraint = np.array([[2,5],
                       [4, 2],
                       [-1, 1]])
constraint_constant = np.array([30,20,0])

In [12]:
result = linprog(-objective, constraint, constraint_constant)

if result.success == True:
    # We use negative sign to get the optimal value (maximized value)
    print('Optimal Value:', -result.fun)
    print(f'(x1, x2): {result.x[0], result.x[1]}')
else:
    print('The problem does not have an optimal solution.')


Optimal Value: 23.333333333333336
(x1, x2): (np.float64(3.333333333333334), np.float64(3.333333333333333))


# Exercise 37.2
A carpenter manufactures $2$ products $A$ and $B$.

Product $A$ generates a profit of $23$ and product $B$ generates a profit of $10$.

It takes $2$ hours for the carpenter to produce $A$ and $0.8$ hours to produce $B$.

Moreover, he can't spend more than $25$ hours a week and the total number of units of $A$ and $B$ should not be greater than $20$.

Find the number of units of $A$ and $B$ he should manufacture in order to maximize his profit.

### Solution
Let $x_{1}$ represent product $A$ and $x_{2}$ represent product $B$. Our problem is $$\begin{align}
\text{max } 23x_{1} &+ 10x_{2} \\
\text{subject to } 2x_{1} & +\:0.8x_{2}  \leq 25 \\
x_{1} & + x_{2} \leq 20 \\
x_{1}&, x_{2} \geq 0
\end{align}$$

In [14]:
obj = np.array([23, 10])
constraint = np.array([[2,0.8],
                       [1, 1]])
constants = np.array([25,20])

In [16]:
soln = linprog(-obj, constraint, constants)

if soln.success == True:
    print(f"The carpenter can maximize his profit by producing {soln.x[0]} units of A and {soln.x[1]} units of B to earn ${-soln.fun}")
else:
    print("There is no solution to the problem")

The carpenter can maximize his profit by producing 7.5 units of A and 12.5 units of B to earn $297.5
